In [1]:
import numpy as np
from copy import deepcopy

To organize the program better, I made a sudoku class: 

In [2]:
class sud:
    def __init__(self,sudarray):
       
        #pospos provides a set of possible values for each position
        self.pospos = dict()
        for r in range(9):
            for c in range(9):
                if sudarray[(r,c)] != 0:
                    self.pospos.update({(r,c):{sudarray[(r,c)]}})
                else:
                    self.pospos.update({(r,c):{1,2,3,4,5,6,7,8,9}})
  
    
    def printsud(self):
        for r in range(9):
            if r%3 == 0:
                print('- - - - - - - - - - - ')
            mattyr = []
            for c in range(9):
                if (c%3 == 0):
                    mattyr.append('|')
                if len(self.pospos[(r,c)]) > 1:
                    mattyr.append('. ')
                else:
                    
                    mattyr.append(str(list(self.pospos[(r,c)])[0]) + ' ')
            mattyr.append('|')
            print(''.join(mattyr))
        print('- - - - - - - - - - - ')
        
    def bigprintsud(self):
        #some helper functions
        def set2string(setty):
            stringy = ""
            for i in setty:
                stringy += str(i)
            return stringy

        def normalize(stringy):
            while len(stringy) < 9:
                stringy+='-'
            return stringy
        
        for r in range(9):
            if r%3 == 0:
                print('----------------------------------------------------------------------------------------------')
            mattyr = []
            for c in range(9):
                if (c%3 == 0):
                    mattyr.append('|')
                mattyr.append(normalize(set2string(self.pospos[(r,c)])))
                mattyr.append(" ")
            mattyr.append('|')
            print(''.join(mattyr))
        print('----------------------------------------------------------------------------------------------')
    
    def legal(self):
        for r in range(9):
            for c in range(9):
                if len(self.pospos[(r,c)]) == 0:
                    return False
        return True
    
    def count_unknowns(self):
        count = 0
        for r in range(9):
            for c in range(9):
                if len(self.pospos[(r,c)]) > 1:
                    count = count + 1
        return count
    
    
    def find_small_pospos(self):
        #finds a cell with the fewest options (but still greater than 1 option)
        #first, ignore the pos with only 1 option
        numb_poss = dict()
        for r in range(9):
            for c in range(9):
                if len(self.pospos[(r,c)]) > 1:
                       numb_poss[(r,c)] = len(self.pospos[(r,c)])
                    
        #then sort the numb_pos dictionary and choose the position with the smallest value
        sorted_pos = sorted(numb_poss, key=numb_poss.__getitem__)
        return sorted_pos[0]

Below are some helper functions for the rules

In [3]:
def expand2powers(k):
    powers = set()
    binaryform = bin(k)
    for i in range(len(binaryform)-2):
        if binaryform[len(binaryform)-i-1] == '1':
            powers = powers.union({i})
    return powers

In [4]:
def union_of_values_row(s,r,set_of_pos):
    u = set()
    for c in set_of_pos:
        u = u.union(s.pospos[(r,c)])
    return u

In [5]:
def union_of_values_col(s,c,set_of_pos):
    u = set()
    for r in set_of_pos:
        u = u.union(s.pospos[(r,c)])
    return u

In [6]:
def union_of_values_section(s,section,set_of_pos):
    u = set()
    sr,sc = section
    for r,c in set_of_pos:
        u = u.union(s.pospos[(3*sr+r,3*sc+c)])
    return u

In [7]:
def rule1(s):
    # If there are k values that cover k elements in a given row, col or section - then no other position
    # in that row, col or section can have any of those k values
    # if applying the rule does nothing to the sudoku state, the function returns False
    change = False
    #check the rows
    for r in range(9):
        for k in range(1,511):
            set_of_pos = expand2powers(k)
            u = union_of_values_row(s,r,set_of_pos)
            if len(u) == len(set_of_pos):
                #update all other positions to not include values in u
                for c in range(9):
                    if c not in set_of_pos:
                        orig = s.pospos[(r,c)].copy()
                        s.pospos[(r,c)] -= u
                        if orig != s.pospos[(r,c)]:
                            change = True
    #check the cols
    for c in range(9):
        for k in range(1,511):
            set_of_pos = expand2powers(k)
            u = union_of_values_col(s,c,set_of_pos)
            if len(u) == len(set_of_pos):
                #update all other positions to not include values in u
                for r in range(9):
                    if r not in set_of_pos:
                        orig = s.pospos[(r,c)].copy()
                        s.pospos[(r,c)] -= u
                        if orig != s.pospos[(r,c)]:
                            change = True
    
    #check the sections
    for sr in range(3):
        for sc in range(3):
            for k in range(1,511):
                set_of_pos = expand2powers(k)
                sectionpos = {(i//3,i%3) for i in set_of_pos}
                u = union_of_values_section(s,(sr,sc),sectionpos)
                if len(u) == len(sectionpos):
                    #update all other positions to not include values in u
                    for r in range(3):
                        for c in range(3):
                            if (r,c) not in sectionpos:
                                orig = s.pospos[(sr*3+r,sc*3+c)].copy()
                                s.pospos[(sr*3+r,sc*3+c)] -= u
                                if orig != s.pospos[(sr*3+r,sc*3+c)]:
                                    change = True
    return change

In [8]:
def rule2(s):
    # if applying the rule does nothing to the sudoku state, the function returns False
    change = False
    #check the rows aginst section
    for r in range(9):
        sr = r//3
        rr = r%3
        secpos = {((rr-1)%3,0),((rr-1)%3,1),((rr-1)%3,2),((rr+1)%3,0),((rr+1)%3,1),((rr+1)%3,2)}
 
        #if a val occurs in a section that intersects a row - and no other occurance of val is possible in the section
        #then no other occurance of val can occur in that row
        #there are three sections to search through
        for sc in range(3):
            vals = union_of_values_row(s,r,{3*sc,3*sc+1,3*sc+2}) - union_of_values_section(s,(sr,sc),secpos)
            if vals:
                for c in {0,1,2,3,4,5,6,7,8}-{3*sc,3*sc+1,3*sc+2}:
                    orig = s.pospos[(r,c)].copy()
                    s.pospos[(r,c)] -= vals
                    if orig != s.pospos[(r,c)]: 
                        change = True
        

    #check the cols aginst section
    for c in range(9):
        sc = c//3
        cc = c%3
        #get positions of other values in section
        secpos = {(0,(cc-1)%3),(1,(cc-1)%3),(2,(cc-1)%3),(0,(cc+1)%3),(1,(cc+1)%3),(2,(cc+1)%3)}
 
        #if a val occurs in a section that intersects a col - and no other occurance of val is possible in the section
        #then no other occurance of val can occur in that col
        #there are three sections to search through
        
        for sr in range(3):
            vals = union_of_values_col(s,c,{3*sr,3*sr+1,3*sr+2}) - union_of_values_section(s,(sr,sc),secpos)
            if vals:
                #remove val from all options along the row except for what's in the section
                for r in {0,1,2,3,4,5,6,7,8}-{3*sr,3*sr+1,3*sr+2}:
                    orig = s.pospos[(r,c)].copy()
                    s.pospos[(r,c)] -= vals
                    if orig != s.pospos[(r,c)]: 
                        change = True
                    
      
    return change

In [9]:
def rule3(s):

    # Suppose you have two rows and two cols where there's a value that cannot occur in any other row along those cols, then that value
    # cannot occur in any other col along those rows (and vice versa)
    # if applying the rule does nothing to the sudoku state, the function returns False
    change = False
    #check the rows aginst section
    for r1 in range(9):
        for r2 in range(9):
            if r1 < r2:
                for c1 in range(9):
                    for c2 in range(9):
                        if c1 < c2:
                            vals = set(range(1,10))-union_of_values_col(s,c1,{0,1,2,3,4,5,6,7,8}-{r1,r2})-union_of_values_col(s,c2,{0,1,2,3,4,5,6,7,8}-{r1,r2})
                                   #then val cannot occur in any other col pos of rows r1,r2
                            if vals: #test if vals is non-empty
                                for c in {0,1,2,3,4,5,6,7,8}-{c1,c2}:
                                    orig = s.pospos[(r1,c)].copy()
                                    s.pospos[(r1,c)] -= vals
                                    if orig != s.pospos[(r1,c)]:
                                        change = True
                                    orig = s.pospos[(r2,c)].copy()
                                    s.pospos[(r2,c)] -= vals
                                    if orig != s.pospos[(r2,c)]:
                                        change = True
                                
                            vals = set(range(1,10))-union_of_values_row(s,r1,{0,1,2,3,4,5,6,7,8}-{c1,c2})-union_of_values_row(s,r2,{0,1,2,3,4,5,6,7,8}-{c1,c2})
                                    #then val cannot occur in any other row pos of cols c1,c2
                             
                            if vals:
                                for r in {0,1,2,3,4,5,6,7,8}-{r1,r2}:
                                    orig = s.pospos[(r,c1)].copy()
                                    s.pospos[(r,c1)] -= vals
                                    if orig != s.pospos[(r,c1)]:
                                        change = True
                                    orig = s.pospos[(r,c2)].copy()
                                    s.pospos[(r,c2)] -= vals
                                    if orig != s.pospos[(r,c2)]:
                                        change = True
                    
        
      
    return change

In [10]:
def solve_puzzle(s):
    stack = []
    stack.append(deepcopy(s))
    
    while stack:
        s = stack.pop()
        while rule1(s) or rule2(s) or rule3(s):
            True

        if s.legal() and s.count_unknowns() == 0:
            print("s is solved!")
            return s
    
        # if the puzzle is still legal, then continue the search by first pushing choices to the stack
        # and then popping one off
        if s.legal():
            small_pos = s.find_small_pospos()
            for x in s.pospos[small_pos]:
                s.pospos[small_pos] = {x}
                stack.append(deepcopy(s))
    
    print("could not find solution")
    return s
    

Fill in the inital sudoku configuration. Do this by creating an array and then call the sud constructor:

In [11]:
sdad = np.zeros((9,9),dtype=np.int8)
sdad[(0,0)] = 8
sdad[(1,2)] = 3
sdad[(1,3)] = 6
sdad[(2,1)] = 7
sdad[(2,4)] = 9
sdad[(2,6)] = 2
sdad[(3,1)] = 5
sdad[(3,5)] = 7
sdad[(4,4)] = 4
sdad[(4,5)] = 5
sdad[(4,6)] = 7
sdad[(5,3)] = 1
sdad[(5,7)] = 3
sdad[(6,2)] = 1
sdad[(6,7)] = 6
sdad[(6,8)] = 8
sdad[(7,2)] = 8
sdad[(7,3)] = 5
sdad[(7,7)] = 1
sdad[(8,1)] = 9
sdad[(8,6)] = 4

suddad = sud(sdad)

In [12]:
suddad.printsud()
suddad.bigprintsud()

- - - - - - - - - - - 
|8 . . |. . . |. . . |
|. . 3 |6 . . |. . . |
|. 7 . |. 9 . |2 . . |
- - - - - - - - - - - 
|. 5 . |. . 7 |. . . |
|. . . |. 4 5 |7 . . |
|. . . |1 . . |. 3 . |
- - - - - - - - - - - 
|. . 1 |. . . |. 6 8 |
|. . 8 |5 . . |. 1 . |
|. 9 . |. . . |4 . . |
- - - - - - - - - - - 
----------------------------------------------------------------------------------------------
|8-------- 123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 |
|123456789 123456789 3-------- |6-------- 123456789 123456789 |123456789 123456789 123456789 |
|123456789 7-------- 123456789 |123456789 9-------- 123456789 |2-------- 123456789 123456789 |
----------------------------------------------------------------------------------------------
|123456789 5-------- 123456789 |123456789 123456789 7-------- |123456789 123456789 123456789 |
|123456789 123456789 123456789 |123456789 4-------- 5-------- |7-------- 123456789 123456789 |
|123456789 123456789 123456789 |1---

In [13]:
s = solve_puzzle(suddad)
s.printsud()

s is solved!
- - - - - - - - - - - 
|8 1 2 |7 5 3 |6 4 9 |
|9 4 3 |6 8 2 |1 7 5 |
|6 7 5 |4 9 1 |2 8 3 |
- - - - - - - - - - - 
|1 5 4 |2 3 7 |8 9 6 |
|3 6 9 |8 4 5 |7 2 1 |
|2 8 7 |1 6 9 |5 3 4 |
- - - - - - - - - - - 
|5 2 1 |9 7 4 |3 6 8 |
|4 3 8 |5 2 6 |9 1 7 |
|7 9 6 |3 1 8 |4 5 2 |
- - - - - - - - - - - 
